# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
import findspark
findspark.init('/home/bahmansheikh/spark-3.1.2-bin-hadoop3.2')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bahman").getOrCreate()

21/12/30 21:42:31 WARN Utils: Your hostname, 1PNG6D3-XPS15 resolves to a loopback address: 127.0.1.1; using 192.168.1.140 instead (on interface wlp108s0)
21/12/30 21:42:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/30 21:42:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [5]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
df.head(1)


[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [11]:
assembler = VectorAssembler(
    inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density'],
    outputCol="features")

In [12]:
output = assembler.transform(df)

In [13]:
final_data = output.select("features",'crew')

In [14]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [15]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               103|
|   mean|  7.48631067961166|
| stddev|3.3336051821376524|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [17]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol='crew')

In [20]:
lrModel = lr.fit(train_data)

21/12/30 21:50:16 WARN Instrumentation: [fcba04fb] regParam is zero, which might cause numerical instability and overfitting.


In [21]:
test_results = lrModel.evaluate(test_data)

In [22]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.21624495057483628|
|   2.369880426874481|
|  0.6986537092929659|
| -0.5029308914720971|
|  0.6134452110864412|
|  0.2680472554519575|
| -0.8597428069235526|
|  0.3319099867554236|
| 0.32764085767951556|
|-0.48362288441434487|
|  0.5351180844435053|
|  -0.587475210822781|
| -0.7751626165344625|
|  0.9743678137267029|
|  0.9743678137267029|
| -0.5969598326908123|
| -0.7581790552433336|
| -1.1441426784265971|
|   2.567959907123445|
| -0.4415816829181818|
+--------------------+
only showing top 20 rows



In [23]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 0.8354437279924499
MSE: 0.6979662226419227


In [24]:
test_results.r2

0.9498535327025746